In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/retail-sales-data/product_hierarchy.csv
/kaggle/input/retail-sales-data/store_cities.csv
/kaggle/input/retail-sales-data/sales.csv/sales.csv


In [2]:
import numpy as np
import pandas as pd
import polars as pl

In [3]:
df=pl.read_csv('/kaggle/input/retail-sales-data/sales.csv/sales.csv')
df

product_id,store_id,date,sales,revenue,stock,price,promo_type_1,promo_bin_1,promo_type_2,promo_bin_2,promo_discount_2,promo_discount_type_2
str,str,str,f64,f64,f64,f64,str,str,str,str,str,str
"""P0001""","""S0002""","""2017-01-02""",0.0,0.0,8.0,6.25,"""PR14""",null,"""PR03""",null,null,null
"""P0001""","""S0012""","""2017-01-02""",1.0,5.3,0.0,6.25,"""PR14""",null,"""PR03""",null,null,null
"""P0001""","""S0013""","""2017-01-02""",2.0,10.59,0.0,6.25,"""PR14""",null,"""PR03""",null,null,null
"""P0001""","""S0023""","""2017-01-02""",0.0,0.0,6.0,6.25,"""PR14""",null,"""PR03""",null,null,null
"""P0001""","""S0025""","""2017-01-02""",0.0,0.0,1.0,6.25,"""PR14""",null,"""PR03""",null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…
"""P0748""","""S0125""","""2019-12-29""",null,null,null,18.9,"""PR14""",null,"""PR03""",null,null,null
"""P0748""","""S0126""","""2019-12-29""",null,null,null,18.9,"""PR14""",null,"""PR03""",null,null,null
"""P0748""","""S0131""","""2019-12-29""",null,null,null,18.9,"""PR14""",null,"""PR03""",null,null,null


In [4]:
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
cols_to_drop = [
    'promo_type_1', 'promo_bin_1',
    'promo_type_2', 'promo_bin_2',
    'promo_discount_2', 'promo_discount_type_2'
]
df = df.drop(cols_to_drop)

In [6]:
import polars as pl

# Identify numeric columns (adjust types if needed)
numeric_cols = [col for col, dtype in df.schema.items() if dtype in (pl.Int64, pl.Float64)]

# Build conditions to filter out outliers using the IQR method for each numeric column
conditions = []
for col in numeric_cols:
    # Calculate Q1 and Q3 for the column
    q1 = df.select(pl.col(col).quantile(0.25)).item()
    q3 = df.select(pl.col(col).quantile(0.75)).item()
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    conditions.append((pl.col(col) >= lower_bound) & (pl.col(col) <= upper_bound))

# Combine conditions (a row is kept only if all numeric columns are within bounds)
combined_condition = conditions[0]
for cond in conditions[1:]:
    combined_condition &= cond

# Filter the DataFrame to remove outliers
df_no_outliers = df.filter(combined_condition)

print(df_no_outliers)
df

shape: (12_560_367, 7)
┌────────────┬──────────┬────────────┬───────┬─────────┬───────┬───────┐
│ product_id ┆ store_id ┆ date       ┆ sales ┆ revenue ┆ stock ┆ price │
│ ---        ┆ ---      ┆ ---        ┆ ---   ┆ ---     ┆ ---   ┆ ---   │
│ str        ┆ str      ┆ str        ┆ f64   ┆ f64     ┆ f64   ┆ f64   │
╞════════════╪══════════╪════════════╪═══════╪═════════╪═══════╪═══════╡
│ P0001      ┆ S0002    ┆ 2017-01-02 ┆ 0.0   ┆ 0.0     ┆ 8.0   ┆ 6.25  │
│ P0001      ┆ S0023    ┆ 2017-01-02 ┆ 0.0   ┆ 0.0     ┆ 6.0   ┆ 6.25  │
│ P0001      ┆ S0025    ┆ 2017-01-02 ┆ 0.0   ┆ 0.0     ┆ 1.0   ┆ 6.25  │
│ P0001      ┆ S0027    ┆ 2017-01-02 ┆ 0.0   ┆ 0.0     ┆ 7.0   ┆ 6.25  │
│ P0001      ┆ S0040    ┆ 2017-01-02 ┆ 0.0   ┆ 0.0     ┆ 19.0  ┆ 6.25  │
│ …          ┆ …        ┆ …          ┆ …     ┆ …       ┆ …     ┆ …     │
│ P0748      ┆ S0125    ┆ 2019-10-31 ┆ 0.0   ┆ 0.0     ┆ 11.0  ┆ 18.9  │
│ P0748      ┆ S0126    ┆ 2019-10-31 ┆ 0.0   ┆ 0.0     ┆ 4.0   ┆ 18.9  │
│ P0748      ┆ S0131    ┆ 20

product_id,store_id,date,sales,revenue,stock,price
str,str,str,f64,f64,f64,f64
"""P0001""","""S0002""","""2017-01-02""",0.0,0.0,8.0,6.25
"""P0001""","""S0012""","""2017-01-02""",1.0,5.3,0.0,6.25
"""P0001""","""S0013""","""2017-01-02""",2.0,10.59,0.0,6.25
"""P0001""","""S0023""","""2017-01-02""",0.0,0.0,6.0,6.25
"""P0001""","""S0025""","""2017-01-02""",0.0,0.0,1.0,6.25
…,…,…,…,…,…,…
"""P0748""","""S0125""","""2019-12-29""",null,null,null,18.9
"""P0748""","""S0126""","""2019-12-29""",null,null,null,18.9
"""P0748""","""S0131""","""2019-12-29""",null,null,null,18.9


In [7]:
from prophet import Prophet
df_pandas = df_pandas.rename(columns={"timestamp": "ds", "value": "y"})

# Fit Prophet model
model = Prophet()
model.fit(df_pandas)

# Make future predictions
future = model.make_future_dataframe(periods=30, freq="D")
forecast = model.predict(future)

print(forecast[["ds", "yhat", "yhat_lower", "yhat_upper"]])

NameError: name 'df_pandas' is not defined